In [ ]:
%%capture
!pip install fasttext

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

import gensim
from gensim.models import KeyedVectors
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import fasttext

import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
class Config:
    BASE_DIR = Path('/content/drive/MyDrive/College/machine_learning_2/submissions/1')
    DATA_DIR = BASE_DIR / 'datasets'
    MODEL_DIR = BASE_DIR / 'models'
    RESULT_DIR = BASE_DIR / 'results'

In [ ]:
df_train = pd.read_csv(Config.DATA_DIR / '20newsgroup_train_preprocessed.csv', index_col='Unnamed: 0')
df_train.head()

,content,target,email,subject,content_cleaned
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,lerxst@wam.umd.edu,WHAT car is this,from wheres my thing subject what car be this ...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,guykuo@carson.u.washington.edu,SI Clock Poll Final Call,from guy kuo subject si clock poll final call ...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,twillis@ec.ecn.purdue.edu,PB questions,from thomas e willis subject pb question organ...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,jgreen@amber,NaN,from joe green subject re weitek p organizatio...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,jcm@head-cfa.harvard.edu,NaN,from jonathan mcdowell subject re shuttle laun...


In [ ]:
df_test = pd.read_csv(Config.DATA_DIR / '20newsgroup_test_preprocessed.csv', index_col='Unnamed: 0')
df_test.head()

,content,target,email,subject,content_cleaned
396,From: ascott@tartarus.uwa.edu.au (Andrew Scott...,12,ascott@tartarus.uwa.edu.au,NaN,from andrew scott subject re how to the disk c...
2788,"From: ""Jon C. R. Bennett"" <jb7m+@andrew.cmu.ed...",11,<jb7m+@andrew.cmu.edu>,NaN,from jon c r bennett subject re ideas on clipp...
4411,From: michael.flood@channel1.com (michael floo...,3,michael.flood@channel1.com,cpu fans,from michael flood subject cpu fan articleid c...
3939,From: jrbd@craycos.com (James Davies)\nSubject...,11,jrbd@craycos.com,NaN,from jam davies subject re clipper consider ha...
4329,From: sichase@csa3.lbl.gov (SCOTT I CHASE)\nSu...,14,sichase@csa3.lbl.gov,NaN,from scott i chase subject re vandalize the sk...


In [ ]:
df_valid = pd.read_csv(Config.DATA_DIR / '20newsgroup_valid_preprocessed.csv', index_col='Unnamed: 0')
df_valid.head()

,content,target,email,subject,content_cleaned
6716,From: krs@allegra.att.com (K. R. Subramanian)\...,1,krs@allegra.att.com,Companies involved with Scientific Visualization,from k r subramanian subject company involve w...
3131,From: lairdb@crash.cts.com (Laird P. Broadfiel...,16,lairdb@crash.cts.com,NaN,from laird p broadfield subject re cnn for sal...
6340,From: galen@picea.CFNR.ColoState.EDU (Galen Wa...,12,galen@picea.CFNR.ColoState.EDU,NaN,from galen watts subject re rf communications ...
6100,Subject: Re: what to do with old 256k SIMMs?\n...,12,cvafymfa@vmsb.is.csupomona.edu,NaN,subject re what to do with old k simms from sr...
3044,From: jlove@ivrit.ra.itd.umich.edu (Jack Love)...,17,jlove@ivrit.ra.itd.umich.edu,NaN,from jack love subject re israeli destruction ...


## CountVectorizer (BoW)

### Hyperparameter tuning max_features parameter in countVectorizer()

In [ ]:
# Split dataframes to X and y
X_train = df_train['content_cleaned']
y_train = df_train['target']

X_valid = df_valid['content_cleaned']
y_valid = df_valid['target']

X_test = df_test['content_cleaned']
y_test = df_test['target']

In [ ]:
# List of max_features values to try
max_feature_list = [200, 500, 1000, 1200, 1500, 2000]

In [ ]:
# Variable to track the best performance
best_f1 = 0
best_max_features = None

# Loop through each value in max_feature_list
for max_features in max_feature_list:
    # Initialize CountVectorizer with the current max_features
    vectorizer = CountVectorizer(max_features=max_features)

    # Transform the training and validation data
    X_train_vect = vectorizer.fit_transform(X_train)
    X_valid_vect = vectorizer.transform(X_valid)

    # Initialize and train Logistic Regression
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train_vect, y_train)

    # Make predictions on the validation set
    y_pred = log_reg.predict(X_valid_vect)

    # Evaluate the model
    f1 = f1_score(y_valid, y_pred, average='macro')

    # Print the results for the current max_features value
    print(f"max_features={max_features} | f1 score: {f1:.4f}")

    # Check if this is the best f1_acc so far
    if f1 > best_f1:
        best_f1 = f1
        best_max_features = max_features

print(f"\nBest max_features: {best_max_features} with f1 score: {best_f1:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


max_features=200 | f1 score: 0.3839
max_features=500 | f1 score: 0.5176
max_features=1000 | f1 score: 0.6099
max_features=1200 | f1 score: 0.6341
max_features=1500 | f1 score: 0.6555
max_features=2000 | f1 score: 0.6787

Best max_features: 2000 with f1 score: 0.6787


In [ ]:
vectorizer = CountVectorizer(max_features=best_max_features)

In [ ]:
train_bow = vectorizer.fit_transform(X_train)
valid_bow = vectorizer.transform(X_valid)
test_bow = vectorizer.transform(X_test)

In [ ]:
X_train_bow = pd.DataFrame(train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_valid_bow = pd.DataFrame(valid_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow = pd.DataFrame(test_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_train_bow.head()

,ability,able,about,above,absolute,absolutely,abuse,ac,academic,accept,...,you,youd,youll,young,your,youre,yours,yourself,youve,zone
0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## TF-IDF

### Hyperparameter tuning max_features for TfidfVectorizer()

In [ ]:
# List of max_features values to try
max_feature_list = [200, 500, 1000, 1200, 1500, 2000]

In [ ]:
# Variable to track the best performance
best_f1 = 0
best_max_features = None

# Loop through each value in max_feature_list
for max_features in max_feature_list:
    # Initialize TfidfVectorizer with the current max_features
    vectorizer = TfidfVectorizer(max_features=max_features)

    # Transform the training and testing data
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Initialize and train Logistic Regression
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train_vect, y_train)

    # Make predictions on the test set
    y_pred = log_reg.predict(X_test_vect)

    # Evaluate the model
    f1 = f1_score(y_test, y_pred, average='macro')

    # Print the results for the current max_features value
    print(f"max_features={max_features} | f1 score: {f1:.4f}")

    # Check if this is the best f1_score so far
    if f1 > best_f1:
        best_f1 = f1
        best_max_features = max_features

print(f"\nBest max_features: {best_max_features} with f1 score: {best_f1:.4f}")

max_features=200 | f1 score: 0.4015
max_features=500 | f1 score: 0.5886
max_features=1000 | f1 score: 0.6845
max_features=1200 | f1 score: 0.7000
max_features=1500 | f1 score: 0.7216
max_features=2000 | f1 score: 0.7462

Best max_features: 2000 with f1 score: 0.7462


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=best_max_features)

In [ ]:
train_tf_idf = tfidf_vectorizer.fit_transform(X_train)
valid_tf_idf = tfidf_vectorizer.transform(X_valid)
test_tf_idf = tfidf_vectorizer.transform(X_test)

In [ ]:
X_train_tf_idf = pd.DataFrame(train_tf_idf.toarray(), columns=vectorizer.get_feature_names_out())
X_valid_tf_idf = pd.DataFrame(valid_tf_idf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tf_idf = pd.DataFrame(test_tf_idf.toarray(), columns=vectorizer.get_feature_names_out())
X_train_tf_idf.head()

,ability,able,about,above,absolute,absolutely,abuse,ac,academic,accept,...,you,youd,youll,young,your,youre,yours,yourself,youve,zone
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.087434,0.0,0.0,0.0,0.062246,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.038573,0.0,0.0,0.0,0.054921,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.031541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024968,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.127109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050310,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.042982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.034024,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


## Word2Vec

In [ ]:
w2v_model_path = Config.DATA_DIR / 'GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(w2v_model_path, binary=True)
print('Done loading Word2Vec model')

Done loading Word2Vec model


In [ ]:
def word2vec_embedding(text, w2v_model, dimension=300):
    """
    Generate a feature vector for an entire document by averaging word vectors.

    Parameters:
        text (str): The document (one full row of content_cleaned).
        w2v_model (gensim.models.KeyedVectors): Pre-trained Word2Vec model.
        dimension (int): Dimensionality of word vectors (default is 300).

    Returns:
        np.ndarray: Averaged word vector representation of the document.
    """
    zero_vector = np.zeros(dimension)
    tokens = text.split()  # Simple whitespace tokenization

    word_vectors = [w2v_model[token] for token in tokens if token in w2v_model]  # Correct way to fetch vectors

    return np.mean(word_vectors, axis=0) if word_vectors else zero_vector  # Handle empty cases

In [ ]:
X_train_word2vec = X_train.apply(lambda txt: word2vec_embedding(txt, w2v_model)).apply(pd.Series)

# Rename columns to something meaningful
X_train_word2vec.columns = [f"w2v_{i}" for i in range(X_train_word2vec.shape[1])]
X_train_word2vec.head()

,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,0.007801,0.007271,0.056149,0.104170,-0.041029,-0.009192,0.039058,-0.065662,0.053402,0.058339,...,-0.072690,0.066802,-0.075329,0.008947,-0.018997,-0.027270,-0.009950,-0.044448,0.009347,-0.056897
1,-0.004851,0.035265,0.039960,0.072817,-0.031159,0.008666,0.015080,-0.072557,0.082459,0.043270,...,-0.026894,0.053776,-0.049920,-0.014080,0.009533,-0.072636,-0.038792,-0.036564,-0.031219,-0.007193
2,0.015247,0.005701,0.019336,0.120636,-0.056817,0.013018,0.018738,-0.067665,0.088968,0.077356,...,-0.036953,0.071148,-0.063797,0.024232,-0.021028,-0.030299,-0.010727,-0.060344,-0.002789,-0.036179
3,0.033739,0.014165,0.014005,0.111028,-0.066243,0.000270,0.031023,-0.057488,0.048414,0.047948,...,-0.044280,0.031794,-0.076340,0.044865,-0.035649,-0.044508,0.001078,-0.069224,0.006071,-0.034711
4,0.001332,0.004079,0.023041,0.097417,-0.092767,0.016627,0.055037,-0.056360,0.072968,0.069730,...,-0.019347,0.058442,-0.052933,0.014080,-0.009634,-0.025273,-0.020354,-0.068289,0.028027,-0.045768


In [ ]:
X_valid_word2vec = X_valid.apply(lambda txt: word2vec_embedding(txt, w2v_model)).apply(pd.Series)
X_valid_word2vec.columns = [f"w2v_{i}" for i in range(X_valid_word2vec.shape[1])]

X_test_word2vec = X_test.apply(lambda txt: word2vec_embedding(txt, w2v_model)).apply(pd.Series)
X_test_word2vec.columns = [f"w2v_{i}" for i in range(X_test_word2vec.shape[1])]

## Doc2Vec

In [ ]:
# tokenize each document in your dataframe
documents = X_train.apply(lambda txt: word_tokenize(txt)).tolist()
len(documents)

11314

In [ ]:
print(documents[0])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'be', 'this', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'line', 'i', 'be', 'wonder', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'the', 'other', 'day', 'it', 'be', 'a', 'door', 'sport', 'car', 'look', 'to', 'be', 'from', 'the', 'late', 's', 'early', 's', 'it', 'be', 'call', 'a', 'bricklin', 'the', 'doors', 'be', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'be', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'be', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'be', 'make', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'look', 'car', 'please', 'email', 'thank', 'il', 'bring', 'to', 'you', 'by', 'your', 'neighborhood', 'least']


In [ ]:
# Tagging each document
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(documents)]
tagged_data[0]

TaggedDocument(words=['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'be', 'this', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'line', 'i', 'be', 'wonder', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'the', 'other', 'day', 'it', 'be', 'a', 'door', 'sport', 'car', 'look', 'to', 'be', 'from', 'the', 'late', 's', 'early', 's', 'it', 'be', 'call', 'a', 'bricklin', 'the', 'doors', 'be', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'be', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'be', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'be', 'make', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'look', 'car', 'please', 'email', 'thank', 'il', 'bring', 'to', 'you', 'by', 'your', 'neighborhood', 'least'], tags=['0']

In [ ]:
# Initialize and train the Doc2Vec model
d2v_model = Doc2Vec(vector_size=300, window=5, min_count=2, workers=4, epochs=10)
d2v_model

In [ ]:
# Build vocabulary from the tagged data
d2v_model.build_vocab(tagged_data)

# Access the vocabulary of the model
vocabulary = d2v_model.wv.key_to_index
print(vocabulary)

{'the': 0, 'be': 1, 'to': 2, 'of': 3, 'a': 4, 'and': 5, 'in': 6, 'i': 7, 'that': 8, 'have': 9, 'it': 10, 'for': 11, 'you': 12, 'from': 13, 'on': 14, 'this': 15, 'not': 16, 'do': 17, 'with': 18, 'as': 19, 'or': 20, 'if': 21, 'but': 22, 'line': 23, 'subject': 24, 'they': 25, 'organization': 26, 'by': 27, 'at': 28, 'can': 29, 'write': 30, 'an': 31, 'will': 32, 'my': 33, 'what': 34, 'all': 35, 'use': 36, 'there': 37, 'would': 38, 'one': 39, 'about': 40, 'we': 41, 'get': 42, 're': 43, 'he': 44, 'your': 45, 'so': 46, 'no': 47, 'say': 48, 'article': 49, 'any': 50, 'me': 51, 'some': 52, 'know': 53, 'who': 54, 'its': 55, 'out': 56, 'think': 57, 'x': 58, 'which': 59, 'people': 60, 'make': 61, 'like': 62, 'go': 63, 'more': 64, 'dont': 65, 'when': 66, 'just': 67, 'university': 68, 'their': 69, 'how': 70, 'up': 71, 'other': 72, 'time': 73, 'only': 74, 'than': 75, 'nntppostinghost': 76, 'them': 77, 'his': 78, 'see': 79, 'also': 80, 'work': 81, 'new': 82, 'then': 83, 'take': 84, 'good': 85, 'im': 86,

In [ ]:
# Train the model
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

In [ ]:
d2v_model.save(os.path.join(Config.MODEL_DIR, "doc2vec_embedding_model.model"))

In [ ]:
saved_d2v_model = Doc2Vec.load(os.path.join(Config.MODEL_DIR, "doc2vec_embedding_model.model"))

In [ ]:
# Get Feature Embeddings
def doc2vec_embedding(text, d2v_model):
    # Preprocess the text and tokenize if necessary
    # Example assumes text is already tokenized, adjust preprocessing if needed
    return d2v_model.infer_vector(text.split())

In [ ]:
X_train_doc2vec = X_train.apply(lambda txt: doc2vec_embedding(txt, saved_d2v_model)).apply(pd.Series)
X_train_doc2vec.columns = [f"d2v_{i}" for i in range(X_train_doc2vec.shape[1])]
X_train_doc2vec

,d2v_0,d2v_1,d2v_2,d2v_3,d2v_4,d2v_5,d2v_6,d2v_7,d2v_8,d2v_9,...,d2v_290,d2v_291,d2v_292,d2v_293,d2v_294,d2v_295,d2v_296,d2v_297,d2v_298,d2v_299
0,0.058226,-0.016757,-0.097241,0.029814,0.202921,-0.120789,-0.147134,0.414580,-0.149258,0.154657,...,-0.124802,0.178962,-0.174475,0.168009,0.257124,0.063796,-0.153964,-0.021093,-0.130848,0.291225
1,0.199822,-0.164836,0.121730,-0.083695,0.030530,-0.309881,-0.043527,-0.234980,-0.218276,0.288485,...,-0.071561,0.050110,0.030963,-0.172786,0.118123,0.217328,0.334614,0.059326,0.033812,0.116749
2,-0.415084,0.223440,0.207867,-0.118879,-0.166404,0.023941,0.302239,0.354493,0.031844,0.317568,...,-0.210473,0.099506,-0.015234,-0.191090,-0.032687,0.295965,0.283132,0.015737,-0.130794,0.498228
3,0.100849,-0.074525,-0.047102,-0.128832,0.078294,-0.057279,-0.115071,0.210700,-0.188603,0.245442,...,0.066012,0.185839,-0.058039,0.156462,0.092567,0.064792,-0.056408,-0.064916,0.056423,0.140134
4,0.048678,0.287919,-0.397682,0.103738,-0.094547,0.042180,0.045834,0.152547,0.081440,0.217798,...,-0.015111,0.001655,0.121931,-0.187768,-0.061273,0.099467,-0.012965,-0.003490,-0.019480,0.305635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,0.133380,0.350382,-0.061951,0.129801,-0.182410,0.228903,0.245070,0.039984,0.454227,-0.357019,...,-0.312907,0.143185,0.253677,-0.319857,-0.050063,0.335674,-0.024307,-0.066517,-0.169054,0.486575
11310,-0.172748,0.016723,0.010492,0.160243,-0.254555,0.266334,-0.097286,0.181040,0.359053,0.202911,...,0.055911,0.068738,0.226901,0.318983,0.095307,0.072947,-0.082490,-0.023974,-0.215925,0.130828
11311,-0.077289,-0.279254,0.112831,-0.214092,0.171853,-0.446115,0.097768,-0.152844,0.272637,0.094835,...,0.274225,0.048398,-0.179278,-0.050668,0.201309,0.130302,-0.160018,0.240560,0.283854,-0.031117
11312,0.052770,-0.009399,-0.297106,0.171077,0.066156,-0.022612,0.091459,0.340909,0.165475,0.275621,...,0.024219,0.028509,0.066403,0.242651,0.302097,-0.027145,-0.166746,-0.292256,0.033744,0.177354


In [ ]:
X_valid_doc2vec = X_valid.apply(lambda txt: doc2vec_embedding(txt, saved_d2v_model)).apply(pd.Series)
X_valid_doc2vec.columns = [f"d2v_{i}" for i in range(X_valid_doc2vec.shape[1])]

X_test_doc2vec = X_test.apply(lambda txt: doc2vec_embedding(txt, saved_d2v_model)).apply(pd.Series)
X_test_doc2vec.columns = [f"d2v_{i}" for i in range(X_test_doc2vec.shape[1])]

## GloVe

In [ ]:
# Function to load GloVe Model
def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

In [ ]:
glove_model = load_glove_model(Config.DATA_DIR / "glove.6B.100d.txt")

In [ ]:
# Get the GloVe embedding for each document
def get_glove_embedding(text, glove_model, vector_size=100):
    # Tokenize the text into words
    tokens = text.split()
    # Initialize an empty embedding (zero vector)
    embedding = np.zeros(vector_size)
    valid_words = 0

    for token in tokens:
        token = token.lower()  # Optionally lowercase the tokens
        if token in glove_model:
            embedding += glove_model[token]
            valid_words += 1

    # If no valid words were found, return a zero vector
    if valid_words > 0:
        embedding /= valid_words
    return embedding

In [ ]:
X_train_glove = X_train.apply(lambda txt: get_glove_embedding(txt, glove_model))
X_train_glove = pd.DataFrame(list(X_train_glove))
X_train_glove.columns = [f"glove_{i}" for i in range(X_train_glove.shape[1])]
X_train_glove.head()

,glove_0,glove_1,glove_2,glove_3,glove_4,glove_5,glove_6,glove_7,glove_8,glove_9,...,glove_90,glove_91,glove_92,glove_93,glove_94,glove_95,glove_96,glove_97,glove_98,glove_99
0,-0.142418,0.213617,0.404310,-0.309287,-0.074278,0.192162,-0.072275,0.282604,-0.021632,0.043619,...,-0.057093,-0.054853,-0.032002,0.070161,-0.465635,-0.058530,-0.179206,-0.273095,0.583156,0.172309
1,-0.263164,0.157382,0.324576,-0.237793,-0.045059,0.136563,-0.063699,0.217542,-0.161846,0.106984,...,-0.050182,0.145797,-0.040129,0.043102,-0.380476,-0.140422,-0.101267,-0.330931,0.467368,0.161903
2,-0.087685,0.210413,0.388596,-0.281718,-0.104794,0.173918,-0.057002,0.154895,-0.064657,-0.035266,...,-0.042456,0.001723,-0.058487,0.055987,-0.420410,-0.086101,-0.145060,-0.270010,0.503386,0.234727
3,-0.137189,0.227155,0.370449,-0.335726,-0.060033,0.080152,-0.023625,0.065831,-0.137578,-0.039882,...,-0.001023,-0.013318,-0.075089,0.071669,-0.349674,-0.046677,-0.098517,-0.322602,0.454288,0.190993
4,-0.141489,0.117340,0.435839,-0.256423,-0.186958,0.104358,-0.070608,0.130507,-0.003023,-0.042054,...,-0.071069,-0.053453,-0.092166,0.035544,-0.351425,-0.007416,-0.070554,-0.292792,0.310907,0.133943


In [ ]:
X_valid_glove = X_valid.apply(lambda txt: get_glove_embedding(txt, glove_model))
X_valid_glove = pd.DataFrame(list(X_valid_glove))
X_valid_glove.columns = [f"glove_{i}" for i in range(X_valid_glove.shape[1])]

X_test_glove = X_test.apply(lambda txt: get_glove_embedding(txt, glove_model))
X_test_glove = pd.DataFrame(list(X_test_glove))
X_test_glove.columns = [f"glove_{i}" for i in range(X_test_glove.shape[1])]

## FastText

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

--2025-03-09 18:58:00--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  54.7MB/s    in 24s     

2025-03-09 18:58:24 (53.0 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
!gunzip cc.en.300.vec.gz

In [ ]:
ft_model = gensim.models.KeyedVectors.load_word2vec_format("cc.en.300.vec", binary=False)

In [ ]:
exp = ft_model["cat"]
exp

array([ 0.0811, -0.0208, -0.0333,  0.2856,  0.1396, -0.1977,  0.1013,
        0.0109, -0.1038,  0.0431, -0.1483, -0.1676, -0.1545, -0.1415,
        0.1274,  0.2279,  0.0769, -0.1387, -0.2019,  0.0153, -0.07  ,
        0.1131,  0.0167,  0.1139,  0.0209, -0.3162,  0.0981, -0.1449,
        0.0995,  0.2212,  0.0222,  0.0675, -0.0647,  0.1127, -0.0257,
       -0.0477,  0.0392,  0.0017, -0.1169, -0.2767, -0.0602,  0.1135,
       -0.113 ,  0.0838, -0.2197,  0.0677,  0.0297, -0.0578, -0.1288,
        0.0936, -0.0628, -0.0858,  0.1738, -0.1004, -0.2897, -0.0184,
        0.0161, -0.0155, -0.1191,  0.2057,  0.2338,  0.1717,  0.0777,
        0.058 , -0.0546, -0.056 ,  0.0791,  0.3294, -0.2105, -0.113 ,
        0.0576,  0.024 ,  0.0492, -0.0012, -0.1901, -0.1345,  0.2146,
       -0.034 ,  0.0105,  0.0774,  0.162 , -0.0519, -0.103 , -0.2243,
       -0.0548,  0.0278,  0.045 ,  0.028 ,  0.0835, -0.0948, -0.086 ,
       -0.3568,  0.1231,  0.0274, -0.0079,  0.0833,  0.1994,  0.0455,
        0.1888,  0.1

In [ ]:
def get_fasttext_embedding(text, ft_model):
    words = text.split()  # Split the text into words
    vectors = []

    for word in words:
        if word in ft_model:
            vectors.append(ft_model[word])

    if vectors:
        # Return the average vector for the text
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if no words were found
        return np.zeros(ft_model.vector_size)


In [ ]:
sentence = "this is a sample sentence"
sentence_embedding = get_fasttext_embedding(sentence, ft_model)
print("Sentence embedding:", sentence_embedding)

Sentence embedding: [ 2.34000059e-03 -9.43599939e-02 -3.67399976e-02  2.41199993e-02
 -5.24200015e-02 -2.45800000e-02  5.44399992e-02  5.34000061e-03
 -4.82000001e-02 -3.30599993e-02 -4.49999981e-02  1.04999999e-02
  3.89199965e-02  3.91199999e-02 -8.40400010e-02  2.08240002e-01
 -9.54000093e-03 -2.96400003e-02 -4.17999923e-03 -1.66199990e-02
 -2.42800005e-02  1.45799965e-02 -1.44200008e-02 -5.43200001e-02
  1.42399995e-02 -5.75999916e-03  2.73600016e-02  2.54399981e-02
 -5.93799949e-02  1.29800007e-01  4.79999930e-04  3.73199992e-02
  1.46200005e-02 -1.19780004e-01 -5.28000016e-03 -1.01200007e-02
  4.70399968e-02 -1.52620003e-01  6.45800009e-02 -3.88399996e-02
 -5.26599996e-02 -3.19998711e-04 -9.13999975e-03  1.28400000e-02
  6.32599965e-02  9.87199992e-02 -4.06400040e-02  5.83599918e-02
  1.73000004e-02  3.45399976e-02  1.05599994e-02 -2.67199986e-02
 -6.81200027e-02 -2.42000017e-02  1.73599999e-02  8.87999963e-03
 -4.91200015e-02 -1.01379991e-01 -1.05579995e-01  9.06000100e-03
 -9.0

In [ ]:
X_train_fasttext = X_train.apply(lambda txt: get_fasttext_embedding(txt, ft_model))

X_train_fasttext = pd.DataFrame(list(X_train_fasttext))
X_train_fasttext.columns = [f"ft_{i}" for i in range(X_train_fasttext.shape[1])]
X_train_fasttext.head()

,ft_0,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_290,ft_291,ft_292,ft_293,ft_294,ft_295,ft_296,ft_297,ft_298,ft_299
0,0.001917,-0.010631,-0.000833,0.045179,-0.052470,-0.005183,-0.012861,-0.001586,0.019473,0.010729,...,0.040371,-0.012130,-0.151398,0.018323,-0.006001,0.014390,0.033597,0.181957,-0.039726,-0.020762
1,0.004658,-0.013980,-0.007707,0.054378,-0.033408,0.008458,0.009996,-0.002053,-0.008513,-0.002466,...,0.044058,0.011347,-0.101804,0.031945,-0.019880,-0.009635,0.020461,0.117116,-0.029500,-0.004950
2,-0.007514,-0.040861,-0.003317,0.044696,-0.046932,-0.006371,-0.016062,0.000425,0.008024,-0.015297,...,0.056046,0.004064,-0.129039,0.010607,-0.005551,0.015989,0.017831,0.148215,-0.051226,-0.026429
3,-0.001969,-0.034006,-0.003502,0.030499,-0.046993,-0.011773,0.003629,-0.018609,0.007212,-0.003684,...,0.067856,-0.006139,-0.132799,0.050791,-0.020194,0.011834,0.016913,0.147791,-0.037426,-0.018607
4,-0.006154,-0.036279,0.000492,0.044005,-0.045885,-0.017797,-0.012349,-0.023803,0.000919,-0.011514,...,0.066847,-0.010858,-0.131297,0.023762,0.000054,0.011336,0.023503,0.166720,-0.027582,-0.001415


In [ ]:
X_valid_fasttext = X_valid.apply(lambda txt: get_fasttext_embedding(txt, ft_model))
X_valid_fasttext = pd.DataFrame(list(X_valid_fasttext))
X_valid_fasttext.columns = [f"ft_{i}" for i in range(X_valid_fasttext.shape[1])]

X_test_fasttext = X_test.apply(lambda txt: get_fasttext_embedding(txt, ft_model))
X_test_fasttext = pd.DataFrame(list(X_test_fasttext))
X_test_fasttext.columns = [f"ft_{i}" for i in range(X_test_fasttext.shape[1])]

In [ ]:
feat_embedding_path = Config.DATA_DIR / 'feature_embeddings'
if not os.path.exists(feat_embedding_path):
    os.makedirs(feat_embedding_path)

In [ ]:
# Save the embeddings to Google drive
embedding_types = ['bow', 'tf_idf', 'word2vec', 'doc2vec', 'glove', 'fasttext']

for embedding in embedding_types:
    globals()[f'X_train_{embedding}'].to_csv(feat_embedding_path / f'X_train_{embedding}.csv')
    globals()[f'X_valid_{embedding}'].to_csv(feat_embedding_path / f'X_valid_{embedding}.csv')
    globals()[f'X_test_{embedding}'].to_csv(feat_embedding_path / f'X_test_{embedding}.csv')

y_train.to_csv(feat_embedding_path / 'y_train.csv')
y_valid.to_csv(feat_embedding_path / 'y_valid.csv')
y_test.to_csv(feat_embedding_path / 'y_test.csv')